In [1]:
import os
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from dotenv import load_dotenv
from professions_list import ALL_PROFESSIONS, PROFESSIONS_LIST
from hh_vacancy_parser_db import HHVacancyParser
from clickhouse_driver import Client

In [2]:
load_dotenv()

CLICKHOUSE_HOST: str = os.getenv("CLICKHOUSE_HOST")
CLICKHOUSE_USER: str = os.getenv("CLICKHOUSE_USER")
CLICKHOUSE_PASSWORD: str = os.getenv("CLICKHOUSE_PASSWORD")
CLICKHOUSE_DATABASE: str = os.getenv("CLICKHOUSE_DATABASE")

In [3]:
clickhouse = Client(host=CLICKHOUSE_HOST, port='9000', user=CLICKHOUSE_USER, password=CLICKHOUSE_PASSWORD, database=CLICKHOUSE_DATABASE, settings={'use_numpy': True})

In [4]:
# clickhouse.execut("""
# CREATE TABLE IF NOT EXISTS hh_vacancies (
#     id String,
#     name Nullable(String),
#     area Nullable(String),
#     url Nullable(String),
#     alternate_url Nullable(String),
#     requirement Nullable(String),
#     responsibility Nullable(String),
#     description Nullable(String),
#     employer Nullable(String),
#     experience Nullable(String),
#     employment Nullable(String),
#     schedule Nullable(String),
#     published_at Nullable(DateTime),
#     created_at Nullable(DateTime),
#     salary_from Nullable(Float64),
#     salary_to Nullable(Float64),
#     salary_currency Nullable(String),
#     salary_gross Nullable(Bool),
#     search_query Nullable(String),
#     area_id Nullable(Int),
#     parsed_date DateTime
# ) ENGINE = MergeTree()
# ORDER BY (parsed_date, id);
# """)

In [5]:
def get_unloaded_position(clickhouse):
    loaded_position = clickhouse.query_dataframe("""
    select distinct search_query
    from hh_vacancies
    """)
    loaded_position = set(loaded_position['search_query'].tolist())
    unloaded_position = list(set(PROFESSIONS_LIST['it_tech']) - loaded_position)
    return sorted(unloaded_position)

In [6]:
unloaded_position = get_unloaded_position(clickhouse)

In [7]:
vacancy_parser = HHVacancyParser(timeout=60, max_retries=5)

In [8]:
for prof in tqdm(unloaded_position):
    df_vacancies = vacancy_parser.load_vacancies(
        search_terms=[prof],
        areas=['Москва'],
        pages=250,
        items_on_page=20,
        delay=2
    )

    vacancy_parser.save_to_clickhouse(df_vacancies, clickhouse)

Сбор вакансий:   0%|                                                       | 0/250 [00:00<?, ?it/s]
Сбор вакансий:   0%| | 1/250 [00:00<00:00, 2794.34it/s, Вакансии=0, Запрос=Android-разработчик, Стр
Сбор вакансий:   1%| | 2/250 [00:04<09:14,  2.24s/it, Вакансии=0, Запрос=Android-разработчик, Стран
Сбор вакансий:   1%| | 2/250 [00:04<09:14,  2.24s/it, Вакансии=20, Запрос=Android-разработчик, Стра
Сбор вакансий:   1%| | 3/250 [00:09<13:44,  3.34s/it, Вакансии=20, Запрос=Android-разработчик, Стра
Сбор вакансий:   1%| | 3/250 [00:09<13:44,  3.34s/it, Вакансии=40, Запрос=Android-разработчик, Стра
Сбор вакансий:   2%| | 4/250 [00:14<15:48,  3.86s/it, Вакансии=40, Запрос=Android-разработчик, Стра
Сбор вакансий:   2%| | 4/250 [00:15<16:22,  4.00s/it, Вакансии=60, Запрос=Android-разработчик, Стра
  2%|█▎                                                             | 1/50 [00:16<13:09, 16.11s/it]

Сбор завершен. Запросов: 4, Ошибок: 0
=== ИТОГИ ===
Всего собрано вакансий: 68
Уникальных вакансий: 68
Распределение по поисковым запросам:
  Android-разработчик: 68
Успешно сохранено 68 записей в ClickHouse таблицу hh_vacancies



Сбор вакансий:   0%|                                                       | 0/250 [00:00<?, ?it/s]
Сбор вакансий:   0%| | 1/250 [00:01<06:11,  1.49s/it, Вакансии=0, Запрос=Angular-разработчик, Стран
  4%|██▌                                                            | 2/50 [00:17<06:01,  7.54s/it]

Сбор завершен. Запросов: 1, Ошибок: 0
=== ИТОГИ ===
Всего собрано вакансий: 10
Уникальных вакансий: 10
Распределение по поисковым запросам:
  Angular-разработчик: 10
Успешно сохранено 10 записей в ClickHouse таблицу hh_vacancies



Сбор вакансий:   0%|                                                       | 0/250 [00:00<?, ?it/s]
Сбор вакансий:   0%| | 1/250 [00:00<00:00, 1455.34it/s, Вакансии=0, Запрос=Automation QA, Страница=
Сбор вакансий:   1%| | 2/250 [00:04<10:06,  2.44s/it, Вакансии=0, Запрос=Automation QA, Страница=1/
Сбор вакансий:   1%| | 2/250 [00:04<10:06,  2.44s/it, Вакансии=20, Запрос=Automation QA, Страница=2
Сбор вакансий:   1%| | 3/250 [00:08<12:34,  3.06s/it, Вакансии=20, Запрос=Automation QA, Страница=2
Сбор вакансий:   1%| | 3/250 [00:08<12:34,  3.06s/it, Вакансии=40, Запрос=Automation QA, Страница=3
Сбор вакансий:   2%| | 4/250 [00:14<17:11,  4.19s/it, Вакансии=40, Запрос=Automation QA, Страница=3
Сбор вакансий:   2%| | 4/250 [00:14<17:11,  4.19s/it, Вакансии=41, Запрос=Automation QA, Страница=4
Сбор вакансий:   2%| | 5/250 [00:21<19:56,  4.88s/it, Вакансии=41, Запрос=Automation QA, Страница=4
Сбор вакансий:   2%| | 5/250 [00:21<19:56,  4.88s/it, Вакансии=41, Запрос=Automation QA, Страница=5

...

Сбор завершен. Запросов: 100, Ошибок: 0
=== ИТОГИ ===
Всего собрано вакансий: 585
Уникальных вакансий: 574
Распределение по поисковым запросам:
  Сетевой инженер: 585
Успешно сохранено 585 записей в ClickHouse таблицу hh_vacancies



Сбор вакансий:   0%|                                                       | 0/250 [00:00<?, ?it/s]
Сбор вакансий:   0%| | 1/250 [00:00<00:00, 3557.51it/s, Вакансии=0, Запрос=Системный администра, Ст
Сбор вакансий:   1%| | 2/250 [00:04<09:56,  2.40s/it, Вакансии=0, Запрос=Системный администра, Стра
Сбор вакансий:   1%| | 2/250 [00:04<09:56,  2.40s/it, Вакансии=0, Запрос=Системный администра, Стра
Сбор вакансий:   1%| | 3/250 [00:09<14:23,  3.50s/it, Вакансии=0, Запрос=Системный администра, Стра
Сбор вакансий:   1%| | 3/250 [00:09<14:23,  3.50s/it, Вакансии=0, Запрос=Системный администра, Стра
Сбор вакансий:   2%| | 4/250 [00:14<16:46,  4.09s/it, Вакансии=0, Запрос=Системный администра, Стра
Сбор вакансий:   2%| | 4/250 [00:14<16:46,  4.09s/it, Вакансии=0, Запрос=Системный администра, Стра
Сбор вакансий:   2%| | 5/250 [00:20<18:37,  4.56s/it, Вакансии=0, Запрос=Системный администра, Стра
Сбор вакансий:   2%| | 5/250 [00:20<18:37,  4.56s/it, Вакансии=0, Запрос=Системный администра, Стра

Сбор завершен. Запросов: 96, Ошибок: 0
=== ИТОГИ ===
Всего собрано вакансий: 258
Уникальных вакансий: 257
Распределение по поисковым запросам:
  Системный администратор: 258
Успешно сохранено 258 записей в ClickHouse таблицу hh_vacancies



Сбор вакансий:   0%|                                                       | 0/250 [00:00<?, ?it/s]
Сбор вакансий:   0%| | 1/250 [00:00<00:00, 1858.35it/s, Вакансии=0, Запрос=Тестировщик, Страница=1/
Сбор вакансий:   1%| | 2/250 [00:04<09:39,  2.34s/it, Вакансии=0, Запрос=Тестировщик, Страница=1/25
Сбор вакансий:   1%| | 2/250 [00:04<09:39,  2.34s/it, Вакансии=0, Запрос=Тестировщик, Страница=2/25
Сбор вакансий:   1%| | 3/250 [00:10<14:56,  3.63s/it, Вакансии=0, Запрос=Тестировщик, Страница=2/25
Сбор вакансий:   1%| | 3/250 [00:10<14:56,  3.63s/it, Вакансии=0, Запрос=Тестировщик, Страница=3/25
Сбор вакансий:   2%| | 4/250 [00:14<16:47,  4.09s/it, Вакансии=0, Запрос=Тестировщик, Страница=3/25
Сбор вакансий:   2%| | 4/250 [00:14<16:47,  4.09s/it, Вакансии=0, Запрос=Тестировщик, Страница=4/25
Сбор вакансий:   2%| | 5/250 [00:20<18:24,  4.51s/it, Вакансии=0, Запрос=Тестировщик, Страница=4/25
Сбор вакансий:   2%| | 5/250 [00:20<18:24,  4.51s/it, Вакансии=0, Запрос=Тестировщик, Страница=5/25

Сбор завершен. Запросов: 67, Ошибок: 0
=== ИТОГИ ===
Всего собрано вакансий: 919
Уникальных вакансий: 916
Распределение по поисковым запросам:
  Тестировщик: 919
Успешно сохранено 919 записей в ClickHouse таблицу hh_vacancies


### 